# Assignment group 2: Network and exploratory data analysis

## Module D _(40 pts)_ An ingredient-based recommender system
In this module we're going to build a recommender system using some recipes data and the Apriori algorithm. These data can be obtained from Kaggle:

- https://www.kaggle.com/kaggle/recipe-ingredients-dataset

and are packaged with the assignment in the following directory:

- `./data/train.json`

In [1]:
#Libraries in use:
from pprint import pprint
from collections import Counter
import json
from itertools import combinations

__D1.__ _(2 pts)_ To start, load the recipe data from `json` format and print the first 5 recipes.

In [2]:
with open('data/train.json', 'r') as f:
    recipes = json.load(f)
pprint(recipes[:5])
print(len(recipes))
#it is a list of dictionaries with cuisines names, ids and ingredients and it has 39774 recipes"

[{'cuisine': 'greek',
  'id': 10259,
  'ingredients': ['romaine lettuce',
                  'black olives',
                  'grape tomatoes',
                  'garlic',
                  'pepper',
                  'purple onion',
                  'seasoning',
                  'garbanzo beans',
                  'feta cheese crumbles']},
 {'cuisine': 'southern_us',
  'id': 25693,
  'ingredients': ['plain flour',
                  'ground pepper',
                  'salt',
                  'tomatoes',
                  'ground black pepper',
                  'thyme',
                  'eggs',
                  'green tomatoes',
                  'yellow corn meal',
                  'milk',
                  'vegetable oil']},
 {'cuisine': 'filipino',
  'id': 20130,
  'ingredients': ['eggs',
                  'pepper',
                  'salt',
                  'mayonaise',
                  'cooking oil',
                  'green chilies',
                  'grilled chicken bre

__D2.__ _(5 pts)_ Next, `from collections import Counter` to write a function called `count_items(recipes)` that counts up the number of recipes that include each `ingredient`, storing each in the counter as a single-element tuple (for downstream convienience), i.e., incrementing like `counts[tuple([ingredient])] +=1`. 

When complete, exhibit this functions utility in application to the `recipes` loaded in __D1__ and print the number of 'candidates' in the output.

In [3]:
CUISINE = tuple(tuple(recipe['ingredients']) for recipe in recipes)

In [4]:
CUISINE[0]

('romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles')

In [5]:
def count_items(recipes):
    #tuple of tuples with ingredients of each recipe
    CUISINE = tuple(tuple(recipe['ingredients']) for recipe in recipes)
    #############################################
    #make a set of all the ingredients:
    ingredients = set() #define a set to avoid duplicates
    for eachCUISINE in CUISINE:
        for eachelement in eachCUISINE:
            ingredients.add(eachelement)
    #############################################
    #counting the recipes with each ingredient:
    counts = Counter()
    for each_ingredient in ingredients:
        for recipe in CUISINE:
            if each_ingredient in recipe:
                counts[tuple([each_ingredient])] +=1
    return counts

In [6]:
candidate_counter = count_items(recipes)

In [7]:
pprint([(x, candidate_counter[x]) for x in list(candidate_counter.keys())[:10]]) 
#for display only(The structure is not what it is diplayed)

[(('broiler',), 2),
 (('Hidden Valley® Original Ranch® Dips Mix',), 2),
 (('jerk sauce',), 6),
 (('chorizo sausage',), 103),
 (('pimenton',), 8),
 (('garlic powder',), 1442),
 (('wondra',), 1),
 (('bertolli vodka sauc made with fresh cream',), 1),
 (('pitted kalamata olives',), 204),
 (('steel-cut oats',), 7)]


In [8]:
print(len(list(candidate_counter.keys()))) #there are 6714 different ingredients in all recipes.

6714


__D3.__ _(5 pts)_ Now, write a function called `store_frequent(candidates, threshold = 25)`, which accepts a `Counter` of `candidates`, i.e., item or itemset counts, and stores only those with count above the determined `threshold` value in a separate counter called `frequent`, which is `return`ed at the end of the function. Apply this function to your output from __D1__ with the default `threshold` value of `25` to exhibit your function's utility, and then print the number of frequent items found.

In [9]:
def store_frequent(candidates, threshold):
    frequent = Counter()
    for eachkey in list(candidates.keys()):
        if candidates[eachkey] >= threshold:
            frequent[eachkey] = candidates[eachkey]
        else:
            pass
    return frequent

In [10]:
frequent_candidates = store_frequent(candidate_counter, 25) #it's a dictionary with frequent ingredients as keys.

In [11]:
pprint([(x, frequent_candidates[x]) for x in list(frequent_candidates.keys())[:10]]) 
#for display only(The structure is not what it is diplayed)

[(('chorizo sausage',), 103),
 (('garlic powder',), 1442),
 (('pitted kalamata olives',), 204),
 (('polenta',), 148),
 (('dark brown sugar',), 319),
 (('red chili peppers',), 644),
 (('chili sauce',), 147),
 (('Belgian endive',), 38),
 (('refried beans',), 250),
 (('artichokes',), 75)]


In [12]:
print(len(list(frequent_candidates.keys()))) #there are 1486 different frequent ingredients in all recipes.

1486


__D4.__ (10 pts) Now, write a function called `get_next(recipes, frequent, threshold = 25)` that accepts the `frequent` items output from the `store_frequent()` function. With these inputs, your function should:

1. create a new `Counter` called `next_candidates`
2. compute the `size` of the itemsets for `next_candidates` from a single key in `frequent`
3. `for` any `recipe` with _at least_ as many ingredients as `size`:
    1. loop over all itemsets of size `size` (see combinations note below)
    2. utilize the apriori principle and subsets of itemsets to count up potentially-frequent candidate itemsets in `next_candidates`
4. `return(next_candidates)` 

__Important__: once your code runs, apply this function to the output of __D3__, report the resulting number of `next_candidates` found, and run `store_frequent` on these to report the number of 2-itemsets that were frequent. Repeat this process to build the 3-itemsets and record in the markdown box any observations on run time for these successive applications. In the response box below reply to the following questions:

- Are we generating more candidates or frequent itemsets as we look at larger sizes? 
- Why would this process become more and more computationally expensive as the size get's larger?
    
Note: to complete this part it is _extremely strongly_ encouraged that you import the `combinations()` function from the `itertools` module. With this, you can execute `combinations(items, k)` to find all combinations of size `k` from a list of `items`.

<font color=blue>To do this computation, we need to compare each candidate against every transaction. If the candidate is contained in a transaction, its support count will be incremented. In general, a dataset that contains k items can potentially generate up to $2^k − 1$ frequent itemsets, excluding the null set. Because k can be very large in many practical applications, the search space of itemsets that need to be explored is exponentially large. This is why doing these calulations is extensively heavy as the set gets larger. Figure below shows the lattice structure that can be used to enumerate the list of all possible itemsets for a set of $\\{a, b, c, d, e\\}$ candidates. [(ref. to here!)](https://www-users.cs.umn.edu/~kumar001/dmbook/ch6.pdf)</font><br >
<img src="./lattice.png"  width="400\" height= "400"/>
<font color=blue>In this problem, the set of all the ingredients is the items and the recipes are the transactions. Each transaction has a subset of the item set. A collection of zero or more items is called the itemset. The Apriori principle is an effective way to eliminate some of the candidate itemsets without counting their support values. Instead of matching each candidate itemset against every transaction, we can reduce the number of comparisons by using more advanced data structure.</font><br>
<img src="./lattice2.png"  width="350\" height= "350"/>
<font color=blue>As the size of the itemsets go up, the run time extends.</font> 

In [51]:
def get_next(recipes, frequent, threshold):
    #########size of a single key###########
    #initial value of size is 1:
    size = len(list(frequent.keys())[0])
    ########################################
    #candidates is the counter for all possible itemsets:
    next_candidates = Counter()
    #go through all recipes
    acceptable_recipes = [x for x in recipes if len(x['ingredients']) >= size+1]
    for recipe in acceptable_recipes:
        for comb in combinations(recipe['ingredients'], size+1):
            if all(tuple(elem) in frequent for elem in combinations(comb, size)):
                comb_as_keys = tuple(x for x in set(sorted(list(comb))))#the order of the ingredients is not important
                next_candidates[comb_as_keys] += 1
    return next_candidates       

In [52]:
get_next_recipe_size2 = get_next(recipes, frequent_candidates, threshold  = 25)

In [53]:
frequent_candidates_size2 = store_frequent(get_next_recipe_size2, 25)

**The print-out below is not sorted:**

In [54]:
pprint([(x, frequent_candidates_size2[x]) for x in list(frequent_candidates_size2.keys())[:10]]) 
#for display only(The structure is not what it is diplayed)

[(('romaine lettuce', 'garlic'), 48),
 (('romaine lettuce', 'pepper'), 42),
 (('purple onion', 'romaine lettuce'), 74),
 (('black olives', 'garlic'), 55),
 (('black olives', 'pepper'), 30),
 (('black olives', 'purple onion'), 34),
 (('grape tomatoes', 'garlic'), 49),
 (('grape tomatoes', 'pepper'), 39),
 (('purple onion', 'grape tomatoes'), 54),
 (('garlic', 'pepper'), 1308)]


In [55]:
get_next_recipe_size3 = get_next(recipes, frequent_candidates_size2, threshold  = 25)

In [56]:
frequent_candidates_size3 = store_frequent(get_next_recipe_size3, 25)

**The print-out below is not sorted:**

In [57]:
pprint([(x, frequent_candidates_size3[x]) for x in list(frequent_candidates_size3.keys())[:10]]) 
#for display only(The structure is not what it is diplayed)

[(('cayenne pepper', 'onions', 'salt'), 74),
 (('onions', 'butter', 'salt'), 78),
 (('onions', 'chili powder', 'oil'), 40),
 (('milk', 'butter', 'salt'), 116),
 (('water', 'chili powder', 'oil'), 26),
 (('sugar', 'butter', 'salt'), 133),
 (('eggs', 'milk', 'vanilla extract'), 30),
 (('salt', 'olive oil', 'pepper'), 244),
 (('salt', 'olive oil', 'flat leaf parsley'), 67),
 (('purple onion', 'ground black pepper', 'salt'), 46)]


__D5.__ (10 pts) Now that we have the pieces to run Apriori/collect frequent itemsets it's time to package the process together, collecting all frequent itemsets up to a particular `size`. To do this, write a function called `train(recipes, size = 4)`, which:

1. initializes two empty dictionaries, `candidates`, and `frequent`;
2. runs the `count_items` and `store_frequent` function, storing output in the `candidates`, and `frequent` dictionaries using the integer `1` as a key;
3. loops over sizes: 2, 3, .., `size` to compute and store the subsequent sizes candidates and frequent itemsets in the same structure as (2), but now utilizing the `get_next` function, instead of `count_items`; and
4. `return`s the `candidates` and `frequent` itemsets.

In [58]:
def train(recipes, size):
    candidates = {}
    frequent = {}
    candidates[1] = count_items(recipes)
    frequent[1] = store_frequent(candidates[1], 25)
    for i in range(2, size+1):
        candidates[i] = get_next(recipes, frequent[i-1], 0)
        frequent[i] = store_frequent(candidates[i], 25)
    return candidates, frequent

In [59]:
candidates, frequent = train(recipes, size = 4)

**These print-outs are not sorted:**

In [60]:
pprint([(x, candidates[1][x]) for x in list(candidates[1].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('broiler',), 2),
 (('Hidden Valley® Original Ranch® Dips Mix',), 2),
 (('jerk sauce',), 6),
 (('chorizo sausage',), 103),
 (('pimenton',), 8)]


In [61]:
pprint([(x, candidates[2][x]) for x in list(candidates[2].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('black olives', 'romaine lettuce'), 7),
 (('romaine lettuce', 'grape tomatoes'), 16),
 (('romaine lettuce', 'garlic'), 48),
 (('romaine lettuce', 'pepper'), 42),
 (('purple onion', 'romaine lettuce'), 74)]


In [62]:
pprint([(x, candidates[3][x]) for x in list(candidates[3].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('romaine lettuce', 'garlic', 'pepper'), 2),
 (('black olives', 'garlic', 'pepper'), 1),
 (('grape tomatoes', 'garlic', 'pepper'), 1),
 (('eggs', 'green tomatoes', 'vegetable oil'), 2),
 (('eggs', 'milk', 'vegetable oil'), 24)]


In [63]:
pprint([(x, candidates[4][x]) for x in list(candidates[4].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('fresh basil', 'olive oil', 'garlic cloves', 'salt'), 10),
 (('sugar', 'large eggs', 'unsalted butter', 'salt'), 21),
 (('onions', 'garlic', 'celery', 'salt'), 5),
 (('onions', 'garlic', 'oil', 'salt'), 6),
 (('onions', 'ginger', 'oil', 'salt'), 4)]


In [64]:
pprint([(x, frequent[1][x]) for x in list(frequent[1].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('chorizo sausage',), 103),
 (('garlic powder',), 1442),
 (('pitted kalamata olives',), 204),
 (('polenta',), 148),
 (('dark brown sugar',), 319)]


In [65]:
pprint([(x, frequent[2][x]) for x in list(frequent[2].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('romaine lettuce', 'garlic'), 48),
 (('romaine lettuce', 'pepper'), 42),
 (('purple onion', 'romaine lettuce'), 74),
 (('black olives', 'garlic'), 55),
 (('black olives', 'pepper'), 30)]


In [66]:
pprint([(x, frequent[3][x]) for x in list(frequent[3].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('cayenne pepper', 'onions', 'salt'), 74),
 (('onions', 'butter', 'salt'), 78),
 (('onions', 'chili powder', 'oil'), 40),
 (('milk', 'butter', 'salt'), 116),
 (('water', 'chili powder', 'oil'), 26)]


In [67]:
pprint([(x, frequent[4][x]) for x in list(frequent[4].keys())[:5]]) 
#for display only(The structure is not what it is diplayed)

[(('water', 'onions', 'olive oil', 'salt'), 28),
 (('all-purpose flour', 'sugar', 'unsalted butter', 'salt'), 29),
 (('baking powder', 'all-purpose flour', 'large eggs', 'salt'), 42),
 (('baking powder', 'all-purpose flour', 'butter', 'salt'), 32),
 (('baking powder', 'all-purpose flour', 'sugar', 'salt'), 33)]


__D5.__ _(8 pts)_ Now that we have our `frequent` itemsets up to `size`, we can utilize them to recommend missing ingredients from ingredient 'baskets' of at most `size - 1`. To do this, write a function called `recommend(basket, frequent)` that does the following: 

1. initializes an empty `recommendations` list
2. loops over all frequent `itemset`s of `size 1 greater than the `basket`
    - if there's one item left from the `itemset` when the `basket` removed, append the remaining item to the `recommendations` list in a tuple, with the number of ocurrences of the itemset in the second position
4. `return` `recommendations`, but sorted from high to low by itemset ocurrence.

Once your code is complete, report the top 10 recommended items to buy for recipe flexibility in the following scenarios:

- `basket = tuple(['butter', 'flour'])`
- `basket = tuple(['soy sauce', 'green onions'])`
- `basket = tuple(['avocado', 'garlic', 'salt'])`

and in the response box below discuss the output and the types of recipes you think the recommender is pointing you to. Does this output seem appropriate? 

Note: your function should additionally respond appropriately if the user requests a recommendation for a basket of size at least as big as the `size` specified in the `train()` function, i.e., it should return an error message gracefully, alerting the user to not having trained on itemsets large enough.

<font color=blue>For example if the basket is flour and butter, the recommender points us to other recipes of probably cakes and desserts. This is true for each basket. </font>

In [68]:
def recommend(basket, frequent):
    initial_recommendations = []
    basket_size = len(basket)
    if basket_size > 3:
        print("Oops! I am not trained for baskets with more than three ingredients :(")
    else:
        for eachitemset in list(frequent[basket_size+1].keys()):
            remainder = tuple([x for x in tuple(eachitemset) if x not in tuple(basket)])
            if len(remainder) == 1:
                initial_recommendations.append((eachitemset, frequent[basket_size+1][eachitemset]))

        recommendations = sorted(initial_recommendations, key = lambda tup:tup[1], reverse = True)
        return recommendations

In [69]:
basket1 = tuple(['butter', 'flour'])
basket2 = tuple(['soy sauce', 'green onions'])
basket3 = tuple(['avocado', 'garlic', 'salt'])
basket4 = tuple(['avocado', 'garlic', 'salt', 'chicken'])

In [70]:
recom_basket1 = recommend(basket1, frequent)

In [71]:
pprint(recom_basket1[:10])

[(('flour', 'butter', 'salt'), 56),
 (('eggs', 'flour', 'butter'), 48),
 (('flour', 'milk', 'butter'), 36),
 (('flour', 'sugar', 'butter'), 28)]


In [72]:
recom_basket2 = recommend(basket2, frequent)

In [73]:
pprint(recom_basket2[:10])

[(('green onions', 'garlic', 'soy sauce'), 66),
 (('green onions', 'soy sauce', 'salt'), 44),
 (('water', 'green onions', 'soy sauce'), 43),
 (('green onions', 'garlic cloves', 'soy sauce'), 37),
 (('eggs', 'green onions', 'soy sauce'), 34),
 (('sesame oil', 'green onions', 'soy sauce'), 32),
 (('green onions', 'ginger', 'soy sauce'), 29),
 (('green onions', 'onions', 'soy sauce'), 26)]


In [74]:
recom_basket3 = recommend(basket3, frequent)

In [75]:
pprint(recom_basket3[:10]) 
#it means that there are no recipe with exactly 4 ingredients where 3 of them are: 'avocado', 'garlic', 'salt'
#which is odd!

[]


In [76]:
recom_basket4 = recommend(basket4, frequent)

Oops! I am not trained for baskets with more than three ingredients :(
